In [4]:
import os
import numpy as np
from urllib.parse import urlencode 
import pandas as pd 
from IPython.display import Image, display, HTML
import astropy.io.votable.tree 
from astropy.io.votable.tree import VOTableFile, Resource, Table, Field
from astropy import units as u 
from astropy.coordinates import SkyCoord
from astropy.time import Time, TimeDelta
from cadcutils import net
import cadctap
from cadctap.core import main_app
from cadctap import CadcTapClient
#from astropy.table import Table
from six import  BytesIO
import glue_jupyter as gj
from ipywidgets import interact, interact_manual 
import csv


params = {'claimedtype' : 'Ia', 'format' : 'csv' }
Catalog_domain ='https://api.sne.space/catalog'
query_str  = urlencode(params)
url = '{}?{}&first'.format(Catalog_domain,query_str)
data = pd.read_csv(url)







def download_subset(Download_subset):
    if(out_data.subsets):
        print(out_data.subsets[1])
#
#
#
def create_a_new_votable(field_dictionary,df):
    print("Creating the votable file for the tmp table.")
    num_row = df.shape[0]  #get total number of rows in the dataframe  
    votable = VOTableFile()
    resource = Resource()
    votable.resources.append(resource)
    table = Table(votable)
    resource.tables.append(table)
    list =[]
    for key, value in field_dictionary.items():
        if(len(value)==2):
            list.append(Field(votable, name = key, datatype= value[0], arraysize = value[1]))
        elif(len(value)==3):
            list.append(Field(votable, name = key, datatype= value[0], arraysize = value[1], xtype = value[2], config={'version_1_2_or_later':True}))#hacky way 
        else:
            list.append(Field(votable, name = key, datatype= value))
                
    table.fields.extend(list)
    table.create_arrays(num_row)
    ### replace the NaN value in satrt_date and end_date with 0.0
    
    for i in range(0,num_row):
        sn = sn_data.iloc[i]
        table.array[i] = (str(sn["event"]),str(sn["alias"]), str(sn["catalog"]), str(sn["claimedtype"]), sn["dec"],
                          str(sn["discoverdate"]),str(sn["discoverer"]),str(sn["download"]),
                          sn["ebv"],str(sn["host"]),str(sn["hostdec"]),sn["hostoffsetang"], sn["hostoffsetdist"],
                          str(sn["hostra"]),str(sn["instruments"]), sn["lumdist"], sn["maxabsmag"], sn["maxappmag"],
                          str(sn["maxdate"]), str(sn["name"]),str(sn["photolink"]),
                          sn["ra" ],sn["radiolink"],  sn["redshift"],str(sn["references"]),sn["sources"], 
                          str(sn["spectralink"]),sn["velocity"],sn["xraylink"],sn["start_date"],sn["end_date"],
                          [sn["start_date"],sn["end_date"]])
    votable.to_xml("tmp/new_votable.xml")


    

@interact
def filter_data(min_redshift =0.2, max_redshift =0.3, event =['SNLS'],days_before_max_date=20, days_after_max_date=100):
    print("set constraints on the supernova catalog table")
    global sn_data
    sn_data = data.copy()
    sn_data = sn_data[(sn_data['redshift']>min_redshift) & (sn_data['redshift']<max_redshift)] 
    sn_data = sn_data[sn_data['event'].str.startswith(event)]
    sn_data = sn_data[sn_data['discoverdate']< sn_data['maxdate']]
    sn_data["ra"] =SkyCoord(sn_data["ra"], sn_data["dec"],unit=(u.hourangle,u.deg)).ra.degree
    sn_data["dec"] = SkyCoord(sn_data["ra"], sn_data["dec"],unit=(u.hourangle,u.deg)).dec.degree
    sn_data["maxdate"] = sn_data["maxdate"].str.replace('/','-')
    print('Number of supernova: {}'.format(len(sn_data)))
    print("Calculating coords and start end date")
    for index, row in sn_data.iterrows():
        if(str(row["maxdate"])!='nan'):
            processed_maxdate = Time(row["maxdate"], format='isot')
            sn_data.at[index, 'start_date'] =  (processed_maxdate - TimeDelta(days_before_max_date,format='jd')).mjd  
            sn_data.at[index, 'end_date'] = (processed_maxdate + TimeDelta(days_after_max_date, format='jd')).mjd
        else: ##set place holder 
            sn_data.at[index, 'start_date'] =  0.0  
            sn_data.at[index, 'end_date'] = 0.1       
        
    sn_data.dropna(subset=['maxdate']) 
        
    field_dictionary = {
        "event":["char", "*"],
        "alias":["char", "*"],
        "catalog":["char", "*"],
        "claimedtype":["char", "*"],
        "dec":"double",
        "discoverdate":["char", "*"],
        "discoverer":["char", "*"],
        "download":["char","*"],
        "ebv":"double" ,        
        "host":["char","*"],  
        "hostdec":["char","*"],  
        "hostoffsetang":"double",  
        "hostoffsetdist":"double", 
        "hostra":["char","*"],       
        "instruments":["char","*"],   
        "lumdist":"double",           
        "maxabsmag":"double",          
        "maxappmag":"double",
        "maxdate" :["char","*"], 
        "name" :["char","*"],
        "photolink":["char","*"],    
        "ra" :"double",
        "radiolink":"double",  
        "redshift":"double",          
        "reference":["char","*"],      
        "sources":"double",              
        "spectralink":["char","*"],      
        "velocity":"double",                
        "xraylink":"double",                
        "start_date":"double",
        "end_date": "double",
        "start_end_date":["double", "2", "interval"]   
    }

    ### create a votable file
    create_a_new_votable(field_dictionary, sn_data)
    ###    
    global out_data
    anonSubject = net.Subject()
    client = CadcTapClient(anonSubject, resource_id ='ivo://cadc.nrc.ca/tap')
    query ="""SELECT calibrationLevel,dataRelease,dataProductType,provenance_name,
        provenance_version,provenance_reference,provenance_producer,provenance_project,
        provenance_keywords,provenance_inputs,metrics_sourceNumberDensity,metrics_background,
        metrics_backgroundStddev,metrics_fluxDensityLimit,metrics_magLimit,position_bounds,
        position_bounds_samples,position_bounds_size,position_resolution,position_sampleSize,
        position_dimension_naxis1,position_dimension_naxis2,position_timeDependent,provenance_lastExecuted,
        quality_flag,position_bounds_points,energy_bounds_samples,energy_bounds_lower,energy_bounds_upper,
        energy_bounds_width,energy_dimension,energy_resolvingPower,energy_sampleSize,energy_freqWidth,
        energy_freqSampleSize,energy_restwav,time_bounds_samples,time_bounds_lower,time_bounds_upper,
        time_bounds_width,time_dimension,time_resolution,time_sampleSize,time_exposure,polarization_dimension,
        energy_emBand,energy_bounds,energy_bandpassName,energy_transition_species,energy_transition_transition,
        time_bounds,polarization_states,sequenceNumber,collection,algorithm_name,type,intent,
        proposal_pi,proposal_keywords,target_standard,target_redshift,target_moving,target_keywords,
        targetPosition_equinox,proposal_project,proposal_title,target_name,target_type,targetPosition_coordsys,
        targetPosition_coordinates_cval1,targetPosition_coordinates_cval2,telescope_geoLocationX,
        telescope_geoLocationY,telescope_geoLocationZ,telescope_keywords,telescope_name,instrument_keywords,
        environment_seeing,environment_humidity,environment_elevation,environment_tau,
        requirements_flag,instrument_name,environment_wavelengthTau,environment_ambientTemp,
        environment_photometric,members,typeCode,event,alias,catalog,claimedtype,dec,discoverdate,discoverer,
        download,ebv,host,hostdec,hostoffsetang,hostoffsetdist,hostra,instruments,lumdist,maxabsmag,maxappmag,
        maxdate,name,photolink,ra,radiolink,redshift,reference,sources,spectralink,velocity,xraylink,start_date,
        end_date,start_end_date FROM caom2.Plane AS Plane
        JOIN caom2.Observation AS Observation ON Plane.obsID = Observation.obsID join tap_upload.tmptable as Tmp on
        (INTERSECTS(Tmp.start_end_date, Plane.time_bounds_samples) = 1
        AND CONTAINS( POINT('ICRS',Tmp.ra, Tmp.dec ), Plane.position_bounds ) = 1)
        WHERE (LOWER(Plane.energy_bandpassName) LIKE 'r%'
        AND collection = 'CFHT'
        AND calibrationLevel >= 2
        AND (Plane.quality_flag IS NULL OR Plane.quality_flag != 'junk'))
        ORDER BY time_bounds_lower ASC"""

    def_table = os.path.join('tmp','new_votable.xml')
    print("fetching data from the database...")
    client.query(query, output_file= "tmp/out_file.csv", response_format= 'csv', tmptable='tmptable:' + def_table, timeout = 30)
    
    ###count the total number of rows returned by the query 
    count_row = len(list(csv.reader(open("tmp/out_file.csv","r+"))))
    print('Total row count: {}'.format(str(count_row)))   
      
    ### display data using glue
    app = gj.jglue()
    out_data = app.load_data('tmp/out_file.csv')
    print("displaying data.")
    scatter_viewer = app.scatter2d(x = 'time_bounds_lower',y = 'time_exposure', data = out_data)
    histogram_viewer = app.histogram1d(x = 'time_exposure', data = out_data)
    interact(download_subset,Download_subset=[out_data.subsets])   

    
    
    
        
    

interactive(children=(FloatSlider(value=0.2, description='min_redshift', max=0.6000000000000001, min=-0.2), Fl…